In [61]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pandas as pd
import math
import fetcher
import datetime
from datetime import timedelta
import warnings; warnings.simplefilter('ignore')

# Sample data for multiple stocks

stocks = ['CDNA', 'VCYT', 'PRCT', 'NTRA', 'GMED', 'TGTX', 'MIRM', 'RARE', 'ALNY', 'LLY', 'UTHR', 'EXEL', 'ADMA', 'ASPN', 'PPC', 'RKLB', 'CR', 'NVDA', 'MPWR', 'AVGO', 'FN', 'TSM', 'CRUS', 'COHR', 'NVMI', 'ITRI', 'ANET', 'STX', 'AAPL', 'CVNA', 'GPI', 'SFM', 'OLLI', 'PDD', 'MELI', 'AMZN', 'BABA', 'BURL', 'HD', 'SG', 'EAT', 'SHAK', 'LTH', 'ATGE', 'LRN', 'HRB', 'BKNG', 'SPHR', 'RCL', 'CDE', 'HL', 'IAG', 'KGC', 'AGI', 'EGO', 'GOLD', 'WPM', 'AEM', 'CRS', 'ATI', 'LPX', 'MOD', 'LII', 'MLI', 'ATMU', 'HA', 'UBER', 'MATX', 'APP', 'ZETA', 'PLTR', 'YOU', 'MNDY', 'TTD', 'CVLT', 'CWAN', 'PANW', 'FICO', 'NOW', 'CRDO', 'NU', 'WIX', 'VRNS', 'FTNT', 'MMYT', 'SPOT', 'META', 'NFLX', 'GOOGL', 'GOOG', 'FIX', 'MTZ', 'PRIM', 'DY', 'VIST', 'VNOM', 'FOUR', 'BCO', 'MHO', 'CCS', 'SKY', 'TOL', 'FTDR', 'MTH', 'TPH', 'PHM', 'SN', 'TBBK', 'BFH', 'WAL', 'GBCI', 'PNFP', 'ALLY', 'STEP', 'TPG', 'KKR', 'JXN', 'URI', 'SLG', 'PIPR', 'LAZ', 'MC', 'VIRT', 'PJT', 'GS', 'JLL', 'KEY', 'BKU', 'ZION', 'UMBF', 'NRG', 'FYBR', 'TPL']


# Calculate the number of rows needed (3 plots per row)
num_stocks = len(stocks)
num_cols = 3
num_rows = math.ceil(num_stocks / num_cols)

# Create subplots with dynamic rows and columns
fig = make_subplots(
    rows=num_rows, 
    cols=num_cols, 
    shared_yaxes=False, 
    subplot_titles=[stock for stock in stocks]
)

startDate = datetime.date(2020, 1, 1)
#endDate = datetime.date(2024, 8, 20)
endDate = datetime.date.today()
# Add candlestick traces and moving averages using a loop
for i, ticker in enumerate(stocks):
    # Calculate the moving averages
    df = fetcher.__download_data( [ticker], startDate, endDate, "US")
    df['Date'] = df.index
    df['EMA_10'] = df['Close'].ewm(span=10, adjust=False).mean()
    df['EMA_21'] = df['Close'].ewm(span=21, adjust=False).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    df['SMA_200'] = df['Close'].rolling(window=200).mean()
    df = df[-63:]

    row = math.floor(i / num_cols) + 1
    col = (i % num_cols) + 1
    
    # Add candlestick trace
    fig.add_trace(go.Candlestick(
        x=df['Date'],
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        showlegend=False
        #name=df['Name'][0]
    ), row=row, col=col)
    
    # Add 10-day EMA trace
    fig.add_trace(go.Scatter(
        x=df['Date'], 
        y=df['EMA_10'], 
        mode='lines',
        line=dict(color='blue', width=1),
        showlegend=False
        #name='10-day EMA'
    ), row=row, col=col)
    
    # Add 21-day EMA trace
    fig.add_trace(go.Scatter(
        x=df['Date'], 
        y=df['EMA_21'], 
        mode='lines',
        line=dict(color='orange', width=1),
        showlegend=False
        #name='21-day EMA'
    ), row=row, col=col)
    
    # Add 50-day SMA trace
    fig.add_trace(go.Scatter(
        x=df['Date'], 
        y=df['SMA_50'], 
        mode='lines',
        line=dict(color='green', width=1),
        showlegend=False
        #name='50-day SMA'
    ), row=row, col=col)

    # Add 200-day SMA trace
    fig.add_trace(go.Scatter(
        x=df['Date'], 
        y=df['SMA_200'], 
        mode='lines',
        line=dict(color='red', width=1),
        showlegend=False
        #name='50-day SMA'
    ), row=row, col=col)

# Update layout for independent zooming and panning and to remove rangesliders
for i in range(1, num_rows * num_cols + 1):
    fig.update_layout({
        f'xaxis{i}_rangeslider_visible': False
    })
alldays =set(df.Date[0]+timedelta(x) for x in range((df.Date[len(df.Date)-1]- df.Date[0]).days))
missing=sorted(set(alldays)-set(df.Date))
fig.update_xaxes(rangebreaks=[dict(values=missing)])
fig.update_layout(
    title="Stock Candlestick Charts with Moving Averages",
    yaxis_title="Price",
    hovermode='closest',  # Ensure hover only shows info for the nearest point
    height=300*num_rows,  # Adjust height dynamically based on the number of rows
    width=1600,
    autosize=True,  # Enable autosizing
    dragmode='pan',  # Enables panning with left-click drag
)

# Enable scroll zoom and remove the box zoom option from the mode bar
config = dict(
    scrollZoom=True,  # Enable zooming with mouse scroll
    displayModeBar=True,  # Show the mode bar for interactivity options
    modeBarButtonsToRemove=['zoom2d'],  # Remove the zoom box tool
)

# Display the plot with the specified configuration
fig.show(config=config)


In [49]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Date', 'EMA_10',
       'EMA_21', 'SMA_50', 'SMA_200'],
      dtype='object')

In [35]:
df

,Open,High,Low,Close,Adj Close,Volume,EMA_10,EMA_21,SMA_50
Date,,,,,,,,,
2024-03-27,170.410004,173.600006,170.110001,173.309998,172.875137,60273300,173.309998,173.309998,NaN
2024-03-28,171.750000,172.229996,170.509995,171.479996,171.049744,65672700,172.977270,173.143634,NaN
2024-04-01,171.190002,171.250000,169.479996,170.029999,169.603378,46240500,172.441402,172.860576,NaN
2024-04-02,169.080002,169.339996,168.229996,168.839996,168.416351,49329500,171.786601,172.495069,NaN
2024-04-03,168.789993,170.679993,168.580002,169.649994,169.224319,47691700,171.398127,172.236426,NaN
...,...,...,...,...,...,...,...,...,...
2024-08-13,219.009995,221.889999,219.009995,221.270004,221.270004,44155300,216.624761,217.574467,215.861201
2024-08-14,220.570007,223.029999,219.699997,221.720001,221.720001,41960600,217.551168,217.951334,216.415001
2024-08-15,224.600006,225.350006,222.759995,224.720001,224.720001,46414000,218.854592,218.566667,217.022401
